In [ ]:
reference_instrument="spi"
subcases_pattern="_10"
osa_version="OSA10.2"
nscw=3
chi2_limit=1.2
ng_sig_limit=3.
systematic_fraction=0.01
freeze_grbm_tem=-1.0

In [ ]:
source_name="Crab"

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt


import numpy as np
import importlib
from astroquery.simbad import Simbad
from astropy import coordinates as coord

from IPython.display import Image
from IPython.display import display 
import xspec
import shutil

import os
import shutil
import oda

from scipy import stats


In [ ]:
if reference_instrument.lower() == 'spi':
    r=oda.evaluate("kb", "http://odahub.io/cc/isgri-oda-spi-reference",
               subcases_pattern=subcases_pattern,
               reference_location=os.getcwd(), 
               source_name=source_name, 
               osa_version=osa_version,
               nscw=nscw,
               _ntries=1)
elif reference_instrument.lower() == 'nustar':
    #r=odakb.evaluate("git@gitlab.astro.unige.ch:integral/cc-workflows/cc-isgri-oda-nustar-reference.git",
    r=oda.evaluate("kb", "http://odahub.io/cc/isgri-oda-nustar-reference",
               subcases_pattern=subcases_pattern,
               reference_location=os.getcwd(), 
               source_name=source_name, 
               osa_version=osa_version,nscw=nscw) 
               #,nbname="isgri-oda-nustar-reference")
else:
    print("Unknown reference instrument")
    raise ValueError

In [ ]:
for k,v in r.items():
    if not k.endswith("_content"):
        print(k,v)
        

In [ ]:
# I add response and arf keywords for easier reading
import astropy.io.fits as pf

if reference_instrument.lower() == 'spi':
    ff=pf.open(r["reference_spec"],  mode='update')
    ff[2].header['RESPFILE']=r["reference_rmf"]
    ff.flush()
    ff.close()
elif reference_instrument.lower() == 'nustar':
    for x in ['A', 'B']:
        
        ff=pf.open(r["reference_spec_"+x],  mode='update')
        ff[1].header['RESPFILE']=r["reference_rmf_"+x]
        ff[1].header['ANCRFILE']=r["reference_arf_"+x]
        ff[1].header['BACKFILE']=r["reference_bkg_"+x]
        ff.flush()
        ff.close()
else:
    print("Undefined reference instrument")
    raise ValueError


ff=pf.open(r["isgri_spec"],  mode='update')
ff[1].header['RESPFILE']=r["isgri_rmf"]
ff[1].header['ANCRFILE']=r["isgri_arf"]
ff.flush()
ff.close()


In [ ]:
import xspec
importlib.reload(xspec)

#init dictionaries
fit_by_lt = {}
fn_by_lt={}

xspec.AllModels.systematic=systematic_fraction   

#Here, I do not care for the test about the low thresholds, I care about the comparison of the cyclotron line energy
#I keep for consistency the loop over one element
for c_emin in [15, 20, 28]: #np.linspace(17,40,5):
    #freeze parameters apart from norm and E_cyc
    
    xspec.AllData.clear()

    if reference_instrument == 'spi':
        model='cflux*grbm'
    elif reference_instrument == 'nustar':
        model='cflux*grbm'
            
    
    m1=xspec.Model(model)
    
    
    if reference_instrument == 'spi':
        xspec.AllData(r['isgri_spec']+" 2:2 " + r["reference_spec"] )
    elif reference_instrument == 'nustar':
        m1(6).link="1"
        m1(7).values = 2.0
        m1(8).values = 1.0
        xspec.AllData(r['isgri_spec']+" 2:2 " + r["reference_spec_A"] + " 3:3 " + r["reference_spec_B"] )
    else:
        raise ValueError
    

    s     = xspec.AllData(1)
    s_ref = xspec.AllData(2)
    
    isgri = xspec.AllModels(1)
    ref   = xspec.AllModels(2)
    
    print(m1.nParameters)
    
    xspec.AllData.ignore('bad')
    xspec.AllData.ignore('500.0-**')
    
    ig="**-%.2f,500.-**"%c_emin
    print(ig)
    s.ignore(ig)
    
    if reference_instrument == 'nustar':
        ig="**-3.0,70.0-**"
        print(ig)
        s_ref.ignore(ig)
        
        s_ref_b=xspec.AllData(3)
        ref_b=xspec.AllModels(3)
        
        s_ref_b.ignore(ig)
    
    #Key for output
    lt_key='%.10lg'%c_emin

        
    #ref.gabs.Strength.link = "%d"%(ref_b.gabs.Strength.index)
    
    
    ref.cflux.lg10Flux.untie()
    ref.cflux.lg10Flux.frozen=False
    
    isgri.cflux.lg10Flux=-8
    ref.cflux.lg10Flux=-8
    
    if reference_instrument == 'nustar':
        ref_b.cflux.lg10Flux.untie()
        ref_b.cflux.lg10Flux.frozen=False
        ref_b.gabs.LineE.link="%d"%(m1.nParameters+ref_b.gabs.LineE.index)
        ref_b.cflux.lg10Flux=-8
    
    
    isgri.cflux.Emin=20.
    isgri.cflux.Emax=80.
    isgri.grbm.norm=1.0
    isgri.grbm.norm.frozen=True
    isgri.grbm.tem=600 if freeze_grbm_tem < 0 else freeze_grbm_tem
    isgri.grbm.tem.frozen=True
        
    xspec.Fit.perform()
    
    isgri.grbm.tem.frozen = freeze_grbm_tem > 0
    xspec.Fit.perform()
    
    if reference_instrument == 'nustar':
        isgri.highecut.foldE.frozen=False
        isgri.highecut.cutoffE.frozen=False
    
    xspec.Fit.perform()
        
    xspec.Fit.error("1.0 max 5.0 1-%d"%(2*m1.nParameters))

    models={}
    
    fit_by_lt[lt_key]=dict(
            emin=c_emin,
            chi2_red=xspec.Fit.statistic/xspec.Fit.dof,                                
            chi2=xspec.Fit.statistic,
            ndof=xspec.Fit.dof,                                    
            models=models,
        )
        
    for m,ss in (isgri, 'isgri'), (ref, 'ref'):
        #initialize dictionaries
        models[ss]={}
        #models[ss]['flux']={}
        
        #fills dictionaries
        for i in range(1,m.nParameters+1): 
            if (not m(i).frozen) and (not bool(m(i).link)):
                #use the name plus position because there could be parameters with same name from multiple 
                #model components (e.g., several gaussians)
                print(m(i).name, "%.2f"%(m(i).values[0]), m(i).frozen,bool(m(i).link) )
                models[ss][m(i).name+"_%02d"%(i)]=[ m(i).values[0], m(i).error[0], m(i).error[1] ]              
        
        


#     for flux_e1, flux_e2 in [(30,80), (80,200)]:
#         xspec.AllModels.calcFlux("{} {} err".format(flux_e1, flux_e2))
#         #print ( xspec.AllData(1).flux)
#         models['isgri']['flux'][(flux_e1, flux_e2)] = xspec.AllData(1).flux
#         models['ref']['flux'][(flux_e1, flux_e2)] = xspec.AllData(2).flux

        

    xspec.Plot.device="/png"
    #xspec.Plot.addCommand("setplot en")
    xspec.Plot.xAxis="keV"
    xspec.Plot("ldata del")
    xspec.Plot.device="/png"
    
    fn="fit_lt%.5lg.png"%c_emin
    
    fn_by_lt[lt_key] = fn
    
    shutil.move("pgplot.png_2", fn)

    _=display(Image(filename=fn,format="png"))



In [ ]:
# marginally more adcanced choice of low thresholds
print("lt\tchi2_r\tprob\tsigmas")
for lt,d in fit_by_lt.items():
    d['nh_prob']=stats.chi2(d['ndof']).sf(d['chi2'])
    d['nh_sig']=stats.norm().isf(d['nh_prob'])
    print("%.1f\t%.2f\t%.2f\t%.2f"%(float(lt), d['chi2_red'], d['nh_prob'], d['nh_sig']) )
    
good_lt = min([p for p in fit_by_lt.items() if p[1]['nh_sig']<ng_sig_limit], key=lambda x:x)
good_lt_next = min([p for p in fit_by_lt.items() if p[1]['chi2_red']<chi2_limit*1.2], key=lambda x:x)


good_lt

In [ ]:
parameter_comparison={}
#compare parameters of best fit
best_result=good_lt[1]
for par_name, par_values_ref in best_result['models']['ref'].items():
    par_values_isgri=best_result['models']['isgri'][par_name]
    isgri_value= par_values_isgri[0]
    ref_value  = par_values_ref[0]
    
    if ref_value > isgri_value:
        err_ref  =  ref_value - par_values_ref[1]
        err_isgri = par_values_isgri[2] - isgri_value
    else:
        err_ref  =  - (ref_value - par_values_ref[2] )
        err_isgri = - (par_values_isgri[1] - isgri_value )

    par_diff_sigmas = (isgri_value - ref_value) /np.sqrt(err_ref**2 + err_isgri**2)
    
    print(par_name+" %.2f +/- %.2f ; %.2f +/- %.2f ; %.1f"%(isgri_value, err_isgri, ref_value, err_ref, par_diff_sigmas))
    
    #We do not compare the normalization with NuSTAR strictly, because of non-simultaneity issues
    if ('log10Flux' in par_name) and (reference_instrument == 'nustar') :
        success=True
    else:
        success = bool(np.abs(par_diff_sigmas) < ng_sig_limit)
    
    
    parameter_comparison[par_name] = dict(
    
        ref = [ref_value, err_ref],
        isgri = [isgri_value, err_isgri ],
        
        significance = np.abs(par_diff_sigmas),
        
        success=success
        
        
        
    )
parameter_comparison


In [ ]:
fn_picked=fn_by_lt[good_lt[0]]
fn_next_picked=fn_by_lt[good_lt_next[0]]

In [ ]:
summary={
    'status': 'OK',
    'emin_good': good_lt,
    'parameter_comparison': parameter_comparison
}

In [ ]:
summary=summary
fit_results=fit_by_lt
good_fit_png=fn_picked
next_good_fit_png=fn_next_picked